In [880]:
from curses import flash
import os
from pickle import TRUE
from re import L
import requests
import validators
import gzip
import shutil
from urllib.parse import urlparse
import random

import sys
import math
import xarray as xa
import numpy as np
from rio_cogeo import cog_validate
import rioxarray

# Mapping
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [881]:
def open_file(input_path):
    return xa.open_dataset(input_file_path, engine="netcdf4", decode_coords='all', decode_times=False)

def copy_lat_lon_data(file, var_name1, var_name2, var_name3, number):
    lat = file[var_name1].data.copy()
    lon = file[var_name2].data.copy()
    data = file[var_name3].data[number].copy()
    return lat, lon, data

def make_new_xarray(lat, lon, data):
#     file = xa.Dataset(
#         data_vars={"flash_extent": (["longitude", "latitude"], data)},
#         coords={["longitude"]: lon, ["latitude"]: lat},
#         attrs={"instrument_ID": "NALMA"}
#     )
    file = xr.Dataset(
        {
            "flash_extent": (["longitude", "latitude"], data),
        },
        coords={
            "longitude": (["longitude"], lon),
            "latitude": (["latitude"], lat),
        },
    )
    return file

def generate_cog(file, variable_name, latitude, longitude, number):
    file = file[variable_name]
    file = file.transpose(latitude, longitude)
    file.rio.set_spatial_dims(x_dim=longitude, y_dim=latitude, inplace=True)
    file.rio.crs
    file.rio.set_crs('epsg:4326')
    #name=f"S2A_20160724_135032_27XVB_B{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{number}.tif"
    name=f"S2A_20160724_135032_27XVB_B{random.randint(0,9)}{number}.tif"
    cog_path=f"/home/asubedi/test_cog/cogs/{name}"
    print(name)
    file.rio.to_raster(rf'{cog_path}', driver='COG')

def delete_row_col(lat, lon, data):
    #finding row with all zeroes    
    zero_row = np.where(np.all(data == 0, axis=1))[0]
    #removing lon with all zeroes
    lon = np.delete(lon, zero_row)
    # Find rows with non-zero elements
    non_zero_rows = np.any(data != 0, axis=1)
    # Filter the array based on non-zero rows
    data = data[non_zero_rows]

    #finding columns with all zeroes
    zero_columns = np.where(np.all(data == 0, axis=0))[0]
    #removing lat with all zeroes
    lat = np.delete(lat, zero_columns)
    # Find rows with non-zero elements
    non_zero_rows = np.any(data != 0, axis=1)
    # Find columns with non-zero elements
    non_zero_columns = np.any(data != 0, axis=0)
    # Filter the array based on non-zero rows and columns
    data = data[non_zero_rows][:, non_zero_columns]
    return lat, lon, data

def flip_lat_data(lat, lon, data):
    lat = lat[::-1]
    data = np.flip(data, axis=1)
    return lat, lon, data

In [882]:
def nalma():
    input_file_path = "/home/asubedi/test_cog/NALMA_20230629_235000_600_10src_0.0109deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def wtlma():
    input_file_path = "/home/asubedi/test_cog/WTLMA_20170531_235000_600_10src_0.0108deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def trmm_lis():
    input_file_path = '/home/asubedi/Desktop/data/raw-files/TRMM-LIS/lis_vhrfc_1998_2013_v01.nc'
    variable_name = "VHRFC_LIS_FRD"
    lat="Latitude"
    lon="Longitude"
    return input_file_path, variable_name, lat, lon

In [884]:
for i in range(10):
    input_file_path, variable_name, latitude, longitude = nalma()
    file = open_file(input_file_path)
    lat, lon, data = copy_lat_lon_data(file, latitude, longitude, variable_name, i)
    lat, lon, data = delete_row_col(lat, lon, data)
    lat, lon, data = flip_lat_data(lat, lon, data)
    file = make_new_xarray(lat, lon ,data)
    generate_cog(file, variable_name, latitude, longitude, i)

S2A_20160724_135032_27XVB_B30.tif
S2A_20160724_135032_27XVB_B81.tif
S2A_20160724_135032_27XVB_B02.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from anot

S2A_20160724_135032_27XVB_B93.tif
S2A_20160724_135032_27XVB_B74.tif
S2A_20160724_135032_27XVB_B45.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from anot

S2A_20160724_135032_27XVB_B96.tif
S2A_20160724_135032_27XVB_B67.tif
S2A_20160724_135032_27XVB_B08.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from anot

S2A_20160724_135032_27XVB_B09.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


In [867]:
file.to_netcdf("/mnt/c/Users/asubedi/Desktop/testing_nalma_new_array4.nc")